In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import mysql.connector
from mysql.connector import Error
import numpy as np

# Load Tables

In [2]:
sales=pd.read_csv("D:\\input Data\\Sales.csv")
customers=pd.read_csv("D:\\input Data\\Customers.csv", encoding='ISO-8859-1')
exchangerates=pd.read_csv("D:\\input Data\\Exchange_Rates.csv")
products=pd.read_csv("D:\\input Data\\Products.csv")
stores=pd.read_csv("D:\\input Data\\Stores.csv")

In [3]:
sales.isnull().sum()

Order Number         0
Line Item            0
Order Date           0
Delivery Date    49719
CustomerKey          0
StoreKey             0
ProductKey           0
Quantity             0
Currency Code        0
dtype: int64

In [4]:
sales.dtypes

Order Number      int64
Line Item         int64
Order Date       object
Delivery Date    object
CustomerKey       int64
StoreKey          int64
ProductKey        int64
Quantity          int64
Currency Code    object
dtype: object

In [6]:
sales['Order Date'] = pd.to_datetime(sales['Order Date'], format='%m/%d/%Y')
sales['Delivery Date'] = pd.to_datetime(sales['Delivery Date'], format='%m/%d/%Y', errors='coerce')
sales['Currency Code'] = sales['Currency Code'].astype(str)

ValueError: time data "01-01-2016" doesn't match format "%m/%d/%Y", at position 0. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.

In [7]:
sales.fillna('Not Available', inplace=True)
sales

,Order Number,Line Item,Order Date,Delivery Date,CustomerKey,StoreKey,ProductKey,Quantity,Currency Code
0,366000,1,01-01-2016,Not Available,265598,10,1304,1,CAD
1,366001,1,01-01-2016,1/13/2016,1269051,0,1048,2,USD
2,366001,2,01-01-2016,1/13/2016,1269051,0,2007,1,USD
3,366002,1,01-01-2016,01-12-2016,266019,0,1106,7,CAD
4,366002,2,01-01-2016,01-12-2016,266019,0,373,1,CAD
...,...,...,...,...,...,...,...,...,...
62879,2243030,1,2/20/2021,Not Available,1216913,43,632,3,USD
62880,2243031,1,2/20/2021,2/24/2021,511229,0,98,4,EUR
62881,2243032,1,2/20/2021,2/23/2021,331277,0,1613,2,CAD
62882,2243032,2,2/20/2021,2/23/2021,331277,0,1717,2,CAD


In [8]:
stores

,StoreKey,Country,State,Square Meters,Open Date
0,1,Australia,Australian Capital Territory,595.0,01-01-2008
1,2,Australia,Northern Territory,665.0,01-12-2008
2,3,Australia,South Australia,2000.0,01-07-2012
3,4,Australia,Tasmania,2000.0,01-01-2010
4,5,Australia,Victoria,2000.0,12-09-2015
...,...,...,...,...,...
62,63,United States,Utah,2000.0,03-06-2008
63,64,United States,Washington DC,1330.0,01-01-2010
64,65,United States,West Virginia,1785.0,01-01-2012
65,66,United States,Wyoming,840.0,01-01-2014


In [9]:
stores.isnull().sum()

StoreKey         0
Country          0
State            0
Square Meters    1
Open Date        0
dtype: int64

In [10]:
stores.dtypes

StoreKey           int64
Country           object
State             object
Square Meters    float64
Open Date         object
dtype: object

In [11]:
stores.fillna(0,inplace=True)

In [12]:
stores.dtypes

StoreKey           int64
Country           object
State             object
Square Meters    float64
Open Date         object
dtype: object

In [13]:
stores['Country']=stores['Country'].astype(str)
stores['State']=stores['State'].astype(str)
stores['Open Date']=pd.to_datetime(stores['Open Date'],format='%m/%d/%Y')

ValueError: time data "01-01-2008" doesn't match format "%m/%d/%Y", at position 0. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.

In [14]:
stores

,StoreKey,Country,State,Square Meters,Open Date
0,1,Australia,Australian Capital Territory,595.0,01-01-2008
1,2,Australia,Northern Territory,665.0,01-12-2008
2,3,Australia,South Australia,2000.0,01-07-2012
3,4,Australia,Tasmania,2000.0,01-01-2010
4,5,Australia,Victoria,2000.0,12-09-2015
...,...,...,...,...,...
62,63,United States,Utah,2000.0,03-06-2008
63,64,United States,Washington DC,1330.0,01-01-2010
64,65,United States,West Virginia,1785.0,01-01-2012
65,66,United States,Wyoming,840.0,01-01-2014


In [15]:
products.isnull().sum()

ProductKey        0
Product Name      0
Brand             0
Color             0
Unit Cost USD     0
Unit Price USD    0
SubcategoryKey    0
Subcategory       0
CategoryKey       0
Category          0
dtype: int64

In [16]:
products['Unit Cost USD'] = products['Unit Cost USD'].replace('[\$,]', '', regex=True).astype(float)
products['Unit Price USD'] = products['Unit Price USD'].replace('[\$,]', '', regex=True).astype(float)

<>:1: SyntaxWarning: invalid escape sequence '\$'
<>:2: SyntaxWarning: invalid escape sequence '\$'
<>:1: SyntaxWarning: invalid escape sequence '\$'
<>:2: SyntaxWarning: invalid escape sequence '\$'
C:\Users\kalpana\AppData\Local\Temp\ipykernel_25920\4131949274.py:1: SyntaxWarning: invalid escape sequence '\$'
  products['Unit Cost USD'] = products['Unit Cost USD'].replace('[\$,]', '', regex=True).astype(float)
C:\Users\kalpana\AppData\Local\Temp\ipykernel_25920\4131949274.py:2: SyntaxWarning: invalid escape sequence '\$'
  products['Unit Price USD'] = products['Unit Price USD'].replace('[\$,]', '', regex=True).astype(float)


In [17]:
products.dtypes

ProductKey          int64
Product Name       object
Brand              object
Color              object
Unit Cost USD     float64
Unit Price USD    float64
SubcategoryKey      int64
Subcategory        object
CategoryKey         int64
Category           object
dtype: object

In [18]:
products

,ProductKey,Product Name,Brand,Color,Unit Cost USD,Unit Price USD,SubcategoryKey,Subcategory,CategoryKey,Category
0,1,Contoso 512MB MP3 Player E51 Silver,Contoso,Silver,6.62,12.99,101,MP4&MP3,1,Audio
1,2,Contoso 512MB MP3 Player E51 Blue,Contoso,Blue,6.62,12.99,101,MP4&MP3,1,Audio
2,3,Contoso 1G MP3 Player E100 White,Contoso,White,7.40,14.52,101,MP4&MP3,1,Audio
3,4,Contoso 2G MP3 Player E200 Silver,Contoso,Silver,11.00,21.57,101,MP4&MP3,1,Audio
4,5,Contoso 2G MP3 Player E200 Red,Contoso,Red,11.00,21.57,101,MP4&MP3,1,Audio
...,...,...,...,...,...,...,...,...,...,...
2512,2513,Contoso Bluetooth Active Headphones L15 Red,Contoso,Red,43.07,129.99,505,Cell phones Accessories,5,Cell phones
2513,2514,Contoso Bluetooth Active Headphones L15 White,Contoso,White,43.07,129.99,505,Cell phones Accessories,5,Cell phones
2514,2515,Contoso In-Line Coupler E180 White,Contoso,White,1.71,3.35,505,Cell phones Accessories,5,Cell phones
2515,2516,Contoso In-Line Coupler E180 Black,Contoso,Black,1.71,3.35,505,Cell phones Accessories,5,Cell phones


In [19]:
customers.isnull().sum()

CustomerKey     0
Gender          0
Name            0
City            0
State Code     10
State           0
Zip Code        0
Country         0
Continent       0
Birthday        0
dtype: int64

In [20]:
customers.dtypes

CustomerKey     int64
Gender         object
Name           object
City           object
State Code     object
State          object
Zip Code       object
Country        object
Continent      object
Birthday       object
dtype: object

In [21]:
def convert_date(date_str):
    try:
        return pd.to_datetime(date_str, format='%m/%d/%Y', errors='coerce').strftime('%m/%d/%Y')
    except ValueError:
        try:
            return pd.to_datetime(date_str, format='%d-%m-%Y', errors='coerce').strftime('%m/%d/%Y')
        except ValueError:
            return date_str 
customers['Birthday'] = customers['Birthday'].apply(convert_date)

In [22]:
customers['State Code'].fillna('Not Available', inplace=True)

C:\Users\kalpana\AppData\Local\Temp\ipykernel_25920\3433429738.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  customers['State Code'].fillna('Not Available', inplace=True)


In [23]:
customers['Birthday'] = pd.to_datetime(customers['Birthday'], format='%m/%d/%Y', errors='coerce')

In [24]:
customers

,CustomerKey,Gender,Name,City,State Code,State,Zip Code,Country,Continent,Birthday
0,301,Female,Lilly Harding,WANDEARAH EAST,SA,South Australia,5523,Australia,Australia,1939-03-07
1,325,Female,Madison Hull,MOUNT BUDD,WA,Western Australia,6522,Australia,Australia,1979-09-27
2,554,Female,Claire Ferres,WINJALLOK,VIC,Victoria,3380,Australia,Australia,1947-05-26
3,786,Male,Jai Poltpalingada,MIDDLE RIVER,SA,South Australia,5223,Australia,Australia,1957-09-17
4,1042,Male,Aidan Pankhurst,TAWONGA SOUTH,VIC,Victoria,3698,Australia,Australia,1965-11-19
...,...,...,...,...,...,...,...,...,...,...
15261,2099600,Female,Denisa Duková,Houston,TX,Texas,77017,United States,North America,1936-03-25
15262,2099618,Male,Justin Solórzano,Mclean,VA,Virginia,22101,United States,North America,1992-02-16
15263,2099758,Male,Svend Petrussen,Wilmington,NC,North Carolina,28405,United States,North America,1937-09-11
15264,2099862,Female,Lorenza Rush,Riverside,CA,California,92501,United States,North America,1937-12-10


In [25]:
exchangerates.dtypes

Date         object
Currency     object
Exchange    float64
dtype: object

In [26]:
exchangerates

,Date,Currency,Exchange
0,01-01-2015,USD,1.0000
1,01-01-2015,CAD,1.1583
2,01-01-2015,AUD,1.2214
3,01-01-2015,EUR,0.8237
4,01-01-2015,GBP,0.6415
...,...,...,...
11210,2/20/2021,USD,1.0000
11211,2/20/2021,CAD,1.2610
11212,2/20/2021,AUD,1.2723
11213,2/20/2021,EUR,0.8238


In [27]:
exchangerates['Date'] = pd.to_datetime(exchangerates['Date'], format='%m/%d/%Y', errors='coerce')

In [28]:
exchangerates

,Date,Currency,Exchange
0,NaT,USD,1.0000
1,NaT,CAD,1.1583
2,NaT,AUD,1.2214
3,NaT,EUR,0.8237
4,NaT,GBP,0.6415
...,...,...,...
11210,2021-02-20,USD,1.0000
11211,2021-02-20,CAD,1.2610
11212,2021-02-20,AUD,1.2723
11213,2021-02-20,EUR,0.8238


In [29]:
sales 

,Order Number,Line Item,Order Date,Delivery Date,CustomerKey,StoreKey,ProductKey,Quantity,Currency Code
0,366000,1,01-01-2016,Not Available,265598,10,1304,1,CAD
1,366001,1,01-01-2016,1/13/2016,1269051,0,1048,2,USD
2,366001,2,01-01-2016,1/13/2016,1269051,0,2007,1,USD
3,366002,1,01-01-2016,01-12-2016,266019,0,1106,7,CAD
4,366002,2,01-01-2016,01-12-2016,266019,0,373,1,CAD
...,...,...,...,...,...,...,...,...,...
62879,2243030,1,2/20/2021,Not Available,1216913,43,632,3,USD
62880,2243031,1,2/20/2021,2/24/2021,511229,0,98,4,EUR
62881,2243032,1,2/20/2021,2/23/2021,331277,0,1613,2,CAD
62882,2243032,2,2/20/2021,2/23/2021,331277,0,1717,2,CAD


In [30]:
customers

,CustomerKey,Gender,Name,City,State Code,State,Zip Code,Country,Continent,Birthday
0,301,Female,Lilly Harding,WANDEARAH EAST,SA,South Australia,5523,Australia,Australia,1939-03-07
1,325,Female,Madison Hull,MOUNT BUDD,WA,Western Australia,6522,Australia,Australia,1979-09-27
2,554,Female,Claire Ferres,WINJALLOK,VIC,Victoria,3380,Australia,Australia,1947-05-26
3,786,Male,Jai Poltpalingada,MIDDLE RIVER,SA,South Australia,5223,Australia,Australia,1957-09-17
4,1042,Male,Aidan Pankhurst,TAWONGA SOUTH,VIC,Victoria,3698,Australia,Australia,1965-11-19
...,...,...,...,...,...,...,...,...,...,...
15261,2099600,Female,Denisa Duková,Houston,TX,Texas,77017,United States,North America,1936-03-25
15262,2099618,Male,Justin Solórzano,Mclean,VA,Virginia,22101,United States,North America,1992-02-16
15263,2099758,Male,Svend Petrussen,Wilmington,NC,North Carolina,28405,United States,North America,1937-09-11
15264,2099862,Female,Lorenza Rush,Riverside,CA,California,92501,United States,North America,1937-12-10


In [31]:
overall=sales

In [32]:
overall=pd.merge(overall,customers,on='CustomerKey',how='inner')

In [33]:
overall.dropna(inplace=True)

In [34]:
overall

,Order Number,Line Item,Order Date,Delivery Date,CustomerKey,StoreKey,ProductKey,Quantity,Currency Code,Gender,Name,City,State Code,State,Zip Code,Country,Continent,Birthday
0,366000,1,01-01-2016,Not Available,265598,10,1304,1,CAD,Male,Tyler Vaught,London,ON,Ontario,N5W 5K6,Canada,North America,1971-03-23
1,366001,1,01-01-2016,1/13/2016,1269051,0,1048,2,USD,Male,Frank Upchurch,Auberry,CA,California,93602,United States,North America,1995-11-20
2,366001,2,01-01-2016,1/13/2016,1269051,0,2007,1,USD,Male,Frank Upchurch,Auberry,CA,California,93602,United States,North America,1995-11-20
3,366002,1,01-01-2016,01-12-2016,266019,0,1106,7,CAD,Female,Joan Roche,Red Deer,AB,Alberta,T4N 2A6,Canada,North America,1962-08-24
4,366002,2,01-01-2016,01-12-2016,266019,0,373,1,CAD,Female,Joan Roche,Red Deer,AB,Alberta,T4N 2A6,Canada,North America,1962-08-24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62879,2243030,1,2/20/2021,Not Available,1216913,43,632,3,USD,Male,Donald Woodie,Garden Grove,CA,California,92643,United States,North America,1939-04-16
62880,2243031,1,2/20/2021,2/24/2021,511229,0,98,4,EUR,Female,Gabriele Thalberg,Stuttgart Frauenkopf,BW,Baden-Württemberg,70184,Germany,Europe,1940-10-03
62881,2243032,1,2/20/2021,2/23/2021,331277,0,1613,2,CAD,Male,William Rochelle,Calgary,AB,Alberta,T2T 0C6,Canada,North America,1993-05-25
62882,2243032,2,2/20/2021,2/23/2021,331277,0,1717,2,CAD,Male,William Rochelle,Calgary,AB,Alberta,T2T 0C6,Canada,North America,1993-05-25


In [35]:
overall=pd.merge(overall,exchangerates,left_on=['Order Date','Currency Code'],right_on=['Date','Currency'],how='inner')

ValueError: You are trying to merge on object and datetime64[ns] columns for key 'Order Date'. If you wish to proceed you should use pd.concat

In [36]:
overall=pd.merge(overall,products,on='ProductKey',how='inner')

In [37]:
overall=pd.merge(overall,stores,on=['StoreKey','Country','State'],how='inner')

In [38]:
overall

,Order Number,Line Item,Order Date,Delivery Date,CustomerKey,StoreKey,ProductKey,Quantity,Currency Code,Gender,...,Brand,Color,Unit Cost USD,Unit Price USD,SubcategoryKey,Subcategory,CategoryKey,Category,Square Meters,Open Date
0,366005,1,01-01-2016,Not Available,844003,33,421,4,EUR,Male,...,Adventure Works,Silver,215.68,469.00,303,Desktops,3,Computers,1540.0,12-09-2015
1,367006,1,01-02-2016,Not Available,693285,18,2445,3,EUR,Female,...,Litware,Green,2.54,4.99,808,Fans,8,Home Appliances,310.0,08-08-2012
2,367010,1,01-02-2016,Not Available,1322959,66,917,1,USD,Male,...,Southridge Video,White,59.31,179.00,308,Computers Accessories,3,Computers,840.0,01-01-2014
3,367010,2,01-02-2016,Not Available,1322959,66,39,1,USD,Male,...,Contoso,Green,99.14,299.23,101,MP4&MP3,1,Audio,840.0,01-01-2014
4,369007,1,01-04-2016,Not Available,571622,20,2510,3,EUR,Male,...,Contoso,White,2.07,4.06,505,Cell phones Accessories,5,Cell phones,1715.0,12/15/2012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6176,2243014,1,2/20/2021,Not Available,1453427,50,423,1,USD,Male,...,Adventure Works,Black,275.46,599.00,303,Desktops,3,Computers,2000.0,03-06-2008
6177,2243014,2,2/20/2021,Not Available,1453427,50,1617,1,USD,Male,...,Contoso,Silver,26.67,57.99,602,Movie DVD,6,"Music, Movies and Audio Books",2000.0,03-06-2008
6178,2243014,3,2/20/2021,Not Available,1453427,50,2506,4,USD,Male,...,Contoso,Pink,2.42,4.74,505,Cell phones Accessories,5,Cell phones,2000.0,03-06-2008
6179,2243021,1,2/20/2021,Not Available,1783631,55,1147,4,USD,Male,...,Fabrikam,Black,301.21,655.00,405,Camcorders,4,Cameras and camcorders,2000.0,12/15/2009


In [39]:
col=list(overall.columns)
col1=[]
for c in col:
    col1.append(c.replace(' ','_').lower())
    
overall.columns=col1

In [40]:
col=list(stores.columns)
col1=[]
for c in col:
    col1.append(c.replace(' ','_').lower())
    
stores.columns=col1

In [41]:
col1

['storekey', 'country', 'state', 'square_meters', 'open_date']

In [42]:
overall.columns

Index(['order_number', 'line_item', 'order_date', 'delivery_date',
       'customerkey', 'storekey', 'productkey', 'quantity', 'currency_code',
       'gender', 'name', 'city', 'state_code', 'state', 'zip_code', 'country',
       'continent', 'birthday', 'product_name', 'brand', 'color',
       'unit_cost_usd', 'unit_price_usd', 'subcategorykey', 'subcategory',
       'categorykey', 'category', 'square_meters', 'open_date'],
      dtype='object')

In [43]:
overall.isnull().sum()

order_number      0
line_item         0
order_date        0
delivery_date     0
customerkey       0
storekey          0
productkey        0
quantity          0
currency_code     0
gender            0
name              0
city              0
state_code        0
state             0
zip_code          0
country           0
continent         0
birthday          0
product_name      0
brand             0
color             0
unit_cost_usd     0
unit_price_usd    0
subcategorykey    0
subcategory       0
categorykey       0
category          0
square_meters     0
open_date         0
dtype: int64

In [44]:
overall.dtypes

order_number               int64
line_item                  int64
order_date                object
delivery_date             object
customerkey                int64
storekey                   int64
productkey                 int64
quantity                   int64
currency_code             object
gender                    object
name                      object
city                      object
state_code                object
state                     object
zip_code                  object
country                   object
continent                 object
birthday          datetime64[ns]
product_name              object
brand                     object
color                     object
unit_cost_usd            float64
unit_price_usd           float64
subcategorykey             int64
subcategory               object
categorykey                int64
category                  object
square_meters            float64
open_date                 object
dtype: object

In [46]:
overall[['order_date','date','currency_code','currency','customerkey']]

KeyError: "['date', 'currency'] not in index"

In [50]:
overall.to_csv('overall.csv')

In [51]:
import pymysql

# Establish a connection to the MySQL server
connection = pymysql.connect(
    host="localhost",
    port=3306,  
    user="root",  
    password="root"
)

try:
    # Create a cursor object
    cursor = connection.cursor()

    # SQL query to create a new database
    database_name = "DataSpark"
    cursor.execute(f"CREATE DATABASE {database_name}")

    print(f"Database '{database_name}' created successfully!")

except pymysql.MySQLError as e:
    print(f"Error occurred: {e}")
    
finally:
    # Close the connection
    cursor.close()
    connection.close()

Database 'DataSpark' created successfully!


In [52]:
import pandas as pd
import pymysql
from sqlalchemy import create_engine
 
host = "localhost"
port = "3306"
username = "root"
password = "root"
database = "DataSpark"
 
engine_string = f"mysql+pymysql://{username}:{password}@{host}:{port}/{database}"
engine = create_engine(engine_string)
 
table_name = "overall"
overall.to_sql(table_name, engine,if_exists='replace',index=False)
print(f"successfully imported {table_name} to sql")

table_name = "customers"
customers.to_sql(table_name, engine,if_exists='replace',index=False)
print(f"successfully imported {table_name} to sql")

table_name = "products"
products.to_sql(table_name, engine,if_exists='replace',index=False)
print(f"successfully imported {table_name} to sql")

table_name = "exchangerates"
exchangerates.to_sql(table_name, engine,if_exists='replace',index=False)
print(f"successfully imported {table_name} to sql")

table_name = "stores"
stores.to_sql(table_name, engine,if_exists='replace',index=False)
print(f"successfully imported {table_name} to sql")

successfully imported overall to sql
successfully imported customers to sql
successfully imported products to sql
successfully imported exchangerates to sql
successfully imported stores to sql
